In [24]:
# import sys
# sys.path.append("/storage/ydong/anaconda3/lib/python3.8/site-packages")
# sys.path.append("/usr/local/lib64/python3.6/site-packages")

import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import pyarrow.parquet as pq
import csv
import sys
import time

In [25]:
infos = pd.read_csv("/storage/ydong/data/gtexRNA/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt",sep = "\t",usecols = ["SAMPID","SMTSD"])

In [26]:
idxList = []
tisList = list(infos["SMTSD"])
for i in range(len(tisList)):
    tisList[i] = tisList[i].replace(" ","")
    tisList[i] = tisList[i].replace("(","_")
    tisList[i] = tisList[i].replace(")","")
# tisTypeList = list(set(tisList))

In [28]:
count = 0
for i in tisList:
    if i == "Lung":
        count +=1

In [29]:
count

867

In [4]:
OUT_PRE = '/gymreklab-tscc/ydong/data/allTis/'
IN_DIR = '/storage/ydong/data/gtexRNA/'

#Read in gtex files
files = [f for f in listdir(IN_DIR) if isfile(join(IN_DIR, f)) and f.split('.')[-1] == 'gct']


In [5]:
try:
    generated = pd.read_csv("junction_Parsed.txt",header = None)
except:
    generated = []

In [6]:
# trim pre-existing folders
# listdir("/gymreklab-tscc/ydong/data/allTis/")

# After preliminary analysis, now we generate tissue specific datasets

In [7]:
# # # create directories
# import os
# for currTis in tisTypeList:
#     OUT_DIR = OUT_PRE + currTis+"/"
# #     os.mkdir(OUT_DIR) 
#     try:
#         os.mkdir(OUT_DIR+"PSI/")    
#     except:
#         print(OUT_DIR)
#     try:
#         os.mkdir(OUT_DIR+"result/")    
#     except:
#         print(OUT_DIR)
        


In [23]:
# deal with naming convention faults made in previous steps
tisDir = os.listdir(OUT_PRE)
for i in tisDir:
    if "(" in i:
        newName = i.replace("(","_")
        newName = newName.replace(")","")
        print(newName)
        os.rename(OUT_PRE + i,OUT_PRE + newName)

In [9]:
currTis = sys.argv[1]

In [7]:
# get the index in junction file
# record it in "samples"
f = open(IN_DIR + files[1], "r")
reader = csv.reader(f, delimiter="\t")
counter = 0
for row in reader:
    counter +=1
    if counter < 3:
        continue
    if counter == 3:
        samples = row
        f.close()
        break
        

In [ ]:
# tissue specific parsing



OUT_DIR = OUT_PRE + currTis.replace(" ","")+"/"


idxList = []
for i in range(len(infos)):
    if ( tisList[i]== currTis ):
        idxList.append(i)
currInfo = infos.iloc[idxList,:]
currInfo = currInfo.set_index("SAMPID")
IDList = list(currInfo.index)

currTisIdx = [0,1]
toRemove = []
for i in IDList:
    if i in samples:
        currTisIdx.append(samples.index(i))
    else:
        toRemove.append(i)

for i in toRemove:
    IDList.remove(i)



# read in the file row by row to avoid loading entire data into memory
f = open(IN_DIR + files[1], "r")
w = open(OUT_DIR + files[1][:-4] + "_Parsed.csv","w")
junctionList = []
reader = csv.reader(f, delimiter="\t")
writer = csv.writer(w,delimiter="\t")
counter = 0
for row in reader:
    counter +=1
    if counter == 3:
        writer.writerow(IDList)
        continue
    if counter < 3:
        continue


    writer.writerow(np.array(row)[currTisIdx])


f.close()
w.close()